<a href="https://colab.research.google.com/github/bashirnubtk/Virtual-CPU-Emulator/blob/main/Final%20submission/Final_Virtual_Cpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import logging

class VirtualCPU:
    def __init__(self):
        """ Initialize CPU with registers, memory, and program counter """
        self.registers = [0] * 16  # 16 General-purpose registers
        self.memory = [0] * 256  # 256-byte memory space
        self.pc = 0  # Program counter
        self.running = True  # CPU running state

        # Logging setup for debugging
        logging.basicConfig(filename="cpu_log.txt", level=logging.INFO, format="%(message)s")

    def fetch(self):
        """ Fetch the next instruction from memory """
        if self.pc < len(self.memory):
            instruction = self.memory[self.pc]
            self.pc += 1
            return instruction
        return None

    def decode(self, instruction):
        """ Decode the fetched instruction """
        return instruction  # Simple pass-through for now

    def execute(self, instruction):
        """ Execute the fetched instruction and log in binary format """
        if instruction is None:
            return

        opcode = instruction[0]
        binary_op = ""

        if opcode == "INPUT":
            reg = instruction[1]
            value = int(input(f"Enter value: "))
            self.registers[reg] = value
            binary_op = f"{bin(value)[2:].zfill(8)}"
            print(f"INPUT: R{reg} = {value} (Binary: {binary_op})")

        elif opcode == "OUTPUT":
            reg = instruction[1]
            value = self.registers[reg]
            binary_op = f"{bin(value)[2:].zfill(8)}"
            print(f"OUTPUT: R{reg} = {value} (Binary: {binary_op})")

        elif opcode == "ADD":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] + self.registers[reg3]
            binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"
            print(f"ADD: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) + R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")

        elif opcode == "SUB":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] - self.registers[reg3]
            binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"
            print(f"SUB: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) - R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")

        elif opcode == "MULT":
            reg1, reg2, reg3 = instruction[1], instruction[2], instruction[3]
            self.registers[reg1] = self.registers[reg2] * self.registers[reg3]
            binary_op = f"{bin(self.registers[reg1])[2:].zfill(8)}"
            print(f"MULT: R{reg1} = R{reg2} ({bin(self.registers[reg2])[2:].zfill(8)}) * R{reg3} ({bin(self.registers[reg3])[2:].zfill(8)}) → {self.registers[reg1]} (Binary: {binary_op})")

        elif opcode == "HALT":
            self.running = False
            print("HALT: Stopping execution")
            binary_op = "00000000"

        else:
            print(f"Unknown opcode: {opcode}")

        # Log binary representation
        logging.info(f"Executed: {instruction} -> Binary: {binary_op}")

    def run(self, program):
        """ Load and execute a program """
        self.memory[:len(program)] = program
        while self.running:
            instruction = self.fetch()
            decoded_instr = self.decode(instruction)
            self.execute(decoded_instr)

        # Final register state
        print(f"Registers: {self.registers}")


# Example Program (With MULT, ADD, SUB)
program = [
    ("INPUT", 0),   # Take input for R0
    ("INPUT", 1),   # Take input for R1
    ("MULT", 2, 0, 1),  # R2 = R0 * R1
    ("OUTPUT", 2),  # Print R2
    ("ADD", 3, 0, 1),  # R3 = R0 + R1
    ("OUTPUT", 3),  # Print R3
    ("SUB", 4, 0, 1),  # R4 = R0 - R1
    ("OUTPUT", 4),  # Print R4
    ("HALT",)       # Stop execution
]

# Run the Virtual CPU
cpu = VirtualCPU()
cpu.run(program)


Enter value: 1
INPUT: R0 = 1 (Binary: 00000001)
Enter value: 3
INPUT: R1 = 3 (Binary: 00000011)
MULT: R2 = R0 (00000001) * R1 (00000011) → 3 (Binary: 00000011)
OUTPUT: R2 = 3 (Binary: 00000011)
ADD: R3 = R0 (00000001) + R1 (00000011) → 4 (Binary: 00000100)
OUTPUT: R3 = 4 (Binary: 00000100)
SUB: R4 = R0 (00000001) - R1 (00000011) → -2 (Binary: 00000b10)
OUTPUT: R4 = -2 (Binary: 00000b10)
HALT: Stopping execution
Registers: [1, 3, 3, 4, -2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
